In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
import xgboost

In [3]:
from sklearn.utils import shuffle
from numpy import sort
from matplotlib import pyplot
from xgboost import plot_importance
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score, classification_report, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve

In [4]:
devices0 = pd.read_csv("devices_featured0_final.csv", low_memory = False)
devices1 = pd.read_csv("devices_featured1_final.csv", low_memory = False)
devices2 = pd.read_csv("devices_featured2_final.csv", low_memory = False)
devices3 = pd.read_csv("devices_featured3_final.csv", low_memory = False)
kaggle = pd.read_csv("devices_featured6_final.csv", low_memory = False)
target = pd.read_csv("target_competencia_ids.csv", low_memory = False)

In [5]:
target_hash = target.ref_hash.str.replace("_sc", "")
target_hash = target_hash.str.replace("_st", "")
target_hash = target_hash.drop_duplicates()
target_hash.to_list()

['1000169251625791246',
 '1000395625957344683',
 '1003027494996471685',
 '1006670001679961544',
 '1007573308966476713',
 '1010070503877148763',
 '1010265377387765028',
 '1010531372912327058',
 '1011610998357271358',
 '1013543838965040946',
 '1014428234021750342',
 '101763096421653092',
 '1017668882534236751',
 '1017958004932387739',
 '1018126962792619181',
 '1021582875881379650',
 '1022093956983448460',
 '1022155574388897311',
 '1024792555410741306',
 '1026825193119998092',
 '1030603044158081433',
 '1030865855693267844',
 '1032938399279723623',
 '1036919443887988520',
 '1037575899869926',
 '1042912100594917197',
 '104412862089502190',
 '1048460865075142817',
 '1048487908287530052',
 '1048782984015604883',
 '1052346027668414186',
 '1052782935269141271',
 '105337483767914646',
 '1053399148291039201',
 '1054881396892383323',
 '1055760083915480820',
 '1056749944537419493',
 '1058525390691423513',
 '1062274422372114651',
 '1069066119870904979',
 '1070673984959261173',
 '1071701264087131067'

In [6]:
devices0_with_target = devices0.loc[devices0['ref_hash'].isin(target_hash.to_list()), :]
devices0_without_target = devices0.loc[~devices0['ref_hash'].isin(target_hash.to_list()), :]

devices1_with_target = devices1.loc[devices1['ref_hash'].isin(target_hash.to_list()), :]
devices1_without_target = devices1.loc[~devices1['ref_hash'].isin(target_hash.to_list()), :]

devices2_with_target = devices2.loc[devices2['ref_hash'].isin(target_hash.to_list()), :]
devices2_without_target = devices2.loc[~devices2['ref_hash'].isin(target_hash.to_list()), :]

devices3_with_target = devices3.loc[devices3['ref_hash'].isin(target_hash.to_list()), :]
devices3_without_target = devices3.loc[~devices3['ref_hash'].isin(target_hash.to_list()), :]

kaggle_with_target = kaggle.loc[kaggle['ref_hash'].isin(target_hash.to_list()), :]

In [7]:
sec_to_conv0 = np.array(devices0['seconds_to_conversion'])
sec_to_conv1 = np.array(devices1['seconds_to_conversion'])
sec_to_conv2 = np.array(devices2['seconds_to_conversion'])
sec_to_conv3 = np.array(devices3['seconds_to_conversion'])
sec_to_auct0 = np.array(devices0['seconds_to_auction'])
sec_to_auct1 = np.array(devices1['seconds_to_auction'])
sec_to_auct2 = np.array(devices2['seconds_to_auction'])
sec_to_auct3 = np.array(devices3['seconds_to_auction'])
data_array_install0 = np.array(devices0.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'auctions_day_1_percent', 'auctions_day_2_percent', 'auctions_day_3_percent', 'total_auctions_normalized', 'mean_diff_between_auctions', 'std_diff_between_auctions']))
data_array_install1 = np.array(devices1.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'auctions_day_1_percent', 'auctions_day_2_percent', 'auctions_day_3_percent', 'total_auctions_normalized', 'mean_diff_between_auctions', 'std_diff_between_auctions']))
data_array_install2 = np.array(devices2.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'auctions_day_1_percent', 'auctions_day_2_percent', 'auctions_day_3_percent', 'total_auctions_normalized', 'mean_diff_between_auctions', 'std_diff_between_auctions']))
data_array_install3 = np.array(devices3.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'auctions_day_1_percent', 'auctions_day_2_percent', 'auctions_day_3_percent', 'total_auctions_normalized', 'mean_diff_between_auctions', 'std_diff_between_auctions']))
data_array_auction0 = np.array(devices0.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'installs_not_implicit_percent', 'installs_implicit_percent', 'installs_day_1', 'installs_day_2', 'installs_day_3', 'total_installs', 'installs_day_1_percent', 'installs_day_2_percent', 'installs_day_3_percent', 'total_installs_normalized']))
data_array_auction1 = np.array(devices1.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'installs_not_implicit_percent', 'installs_implicit_percent', 'installs_day_1', 'installs_day_2', 'installs_day_3', 'total_installs', 'installs_day_1_percent', 'installs_day_2_percent', 'installs_day_3_percent', 'total_installs_normalized']))
data_array_auction2 = np.array(devices2.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'installs_not_implicit_percent', 'installs_implicit_percent', 'installs_day_1', 'installs_day_2', 'installs_day_3', 'total_installs', 'installs_day_1_percent', 'installs_day_2_percent', 'installs_day_3_percent', 'total_installs_normalized']))
data_array_auction3 = np.array(devices3.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'installs_not_implicit_percent', 'installs_implicit_percent', 'installs_day_1', 'installs_day_2', 'installs_day_3', 'total_installs', 'installs_day_1_percent', 'installs_day_2_percent', 'installs_day_3_percent', 'total_installs_normalized']))

data_kaggle_install = np.array(kaggle.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'auctions_day_1_percent', 'auctions_day_2_percent', 'auctions_day_3_percent', 'total_auctions_normalized', 'mean_diff_between_auctions', 'std_diff_between_auctions']))
data_kaggle_auction = np.array(kaggle.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'installs_not_implicit_percent', 'installs_implicit_percent', 'installs_day_1', 'installs_day_2', 'installs_day_3', 'total_installs', 'installs_day_1_percent', 'installs_day_2_percent', 'installs_day_3_percent', 'total_installs_normalized']))

data_devices0_with_target_install = np.array(devices0_with_target.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'auctions_day_1_percent', 'auctions_day_2_percent', 'auctions_day_3_percent', 'total_auctions_normalized', 'mean_diff_between_auctions', 'std_diff_between_auctions']))
data_devices1_with_target_install = np.array(devices1_with_target.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'auctions_day_1_percent', 'auctions_day_2_percent', 'auctions_day_3_percent', 'total_auctions_normalized', 'mean_diff_between_auctions', 'std_diff_between_auctions']))
data_devices2_with_target_install = np.array(devices2_with_target.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'auctions_day_1_percent', 'auctions_day_2_percent', 'auctions_day_3_percent', 'total_auctions_normalized', 'mean_diff_between_auctions', 'std_diff_between_auctions']))
data_devices3_with_target_install = np.array(devices3_with_target.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'auctions_day_1_percent', 'auctions_day_2_percent', 'auctions_day_3_percent', 'total_auctions_normalized', 'mean_diff_between_auctions', 'std_diff_between_auctions']))
data_kaggle_with_target_install = np.array(kaggle_with_target.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'auctions_day_1_percent', 'auctions_day_2_percent', 'auctions_day_3_percent', 'total_auctions_normalized', 'mean_diff_between_auctions', 'std_diff_between_auctions']))

data_devices0_with_target_auction = np.array(devices0_with_target.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'installs_not_implicit_percent', 'installs_implicit_percent', 'installs_day_1', 'installs_day_2', 'installs_day_3', 'total_installs', 'installs_day_1_percent', 'installs_day_2_percent', 'installs_day_3_percent', 'total_installs_normalized']))
data_devices1_with_target_auction = np.array(devices1_with_target.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'installs_not_implicit_percent', 'installs_implicit_percent', 'installs_day_1', 'installs_day_2', 'installs_day_3', 'total_installs', 'installs_day_1_percent', 'installs_day_2_percent', 'installs_day_3_percent', 'total_installs_normalized']))
data_devices2_with_target_auction = np.array(devices2_with_target.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'installs_not_implicit_percent', 'installs_implicit_percent', 'installs_day_1', 'installs_day_2', 'installs_day_3', 'total_installs', 'installs_day_1_percent', 'installs_day_2_percent', 'installs_day_3_percent', 'total_installs_normalized']))
data_devices3_with_target_auction = np.array(devices3_with_target.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'installs_not_implicit_percent', 'installs_implicit_percent', 'installs_day_1', 'installs_day_2', 'installs_day_3', 'total_installs', 'installs_day_1_percent', 'installs_day_2_percent', 'installs_day_3_percent', 'total_installs_normalized']))
data_kaggle_with_target_auction = np.array(kaggle_with_target.drop(columns=['seconds_to_conversion', 'ref_hash', 'seconds_to_auction', 'installs_not_implicit_percent', 'installs_implicit_percent', 'installs_day_1', 'installs_day_2', 'installs_day_3', 'total_installs', 'installs_day_1_percent', 'installs_day_2_percent', 'installs_day_3_percent', 'total_installs_normalized']))

In [8]:
x_train0, x_test0, y_train0, y_test0 = train_test_split(data_array_install0, sec_to_conv0, test_size = 0.30, random_state = 1)
x_train1, x_test1, y_train1, y_test1 = train_test_split(data_array_install1, sec_to_conv1, test_size = 0.30, random_state = 2)
x_train2, x_test2, y_train2, y_test2 = train_test_split(data_array_install2, sec_to_conv2, test_size = 0.30, random_state = 3)
x_train3, x_test3, y_train3, y_test3 = train_test_split(data_array_install3, sec_to_conv3, test_size = 0.30, random_state = 4)

x_train4, x_test4, y_train4, y_test4 = train_test_split(data_array_auction0, sec_to_auct0, test_size = 0.30, random_state = 5)
x_train5, x_test5, y_train5, y_test5 = train_test_split(data_array_auction1, sec_to_auct1, test_size = 0.30, random_state = 6)
x_train6, x_test6, y_train6, y_test6 = train_test_split(data_array_auction2, sec_to_auct2, test_size = 0.30, random_state = 7)
x_train7, x_test7, y_train7, y_test7 = train_test_split(data_array_auction3, sec_to_auct3, test_size = 0.30, random_state = 8)

In [9]:
def logisticReg(x_train, x_test, y_train, y_test):
    logReg = LogisticRegression(solver='sag', random_state=1)
    logReg.fit(x_train, y_train)
    y_pred = logReg.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))

In [24]:
def knn(k, x_train, x_test, y_train, y_test):
    knn = KNeighborsRegressor(n_neighbors=k, metric='manhattan')
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))

In [11]:
def decisionTree(max_depth, max_features, x_train, x_test, y_train, y_test):
    dt = DecisionTreeRegressor(max_depth=max_depth, max_features=max_features)
    dt.fit(x_train, y_train)
    y_pred = dt.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))

In [12]:
def xgboost(max_depth, n_estimators, learning_rate, subsample, colsample_bytree, x_train, x_test, y_train, y_test):
    xgb = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate, subsample=subsample, colsample_bytree=colsample_bytree)
    xgb.fit(x_train, y_train)
    y_pred = xgb.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))

In [13]:
def adaboost(nEstimators, learningRate, x_train, x_test, y_train, y_test):
    ada = AdaBoostRegressor(n_estimators=nEstimators, learning_rate=learningRate, random_state=0)
    ada.fit(x_train, y_train)
    y_pred = ada.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))

In [14]:
def bagging(baseEstimator, nEstimators, learningRate, max_feature, bootstrap, x_train, x_test, y_train, y_test):
    bag = BaggingRegressor(base_estimator=baseEstimator, n_estimators=nEstimators, max_features=max_feature, bootstrap=bootstrap)
    bag.fit(x_train, y_train)
    y_pred = bag.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))

In [15]:
def randomForest(n_estimators, max_depth, x_train, x_test, y_train, y_test):
    rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
    rf.fit(x_train, y_train)
    y_pred = rf.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))

In [19]:
knn_grid_param = {'n_neighbors': [1,3,5,7],
               'algorithm': ['auto'],
               'metric': ['manhattan', 'euclidean', 'minkowski']}

In [20]:
knn = KNeighborsRegressor()
knn_gs = GridSearchCV(estimator=knn, param_grid=knn_grid_param, scoring="neg_mean_squared_error", n_jobs=-1)

In [21]:
knn_gs.fit(x_train0, y_train0)

/home/martin/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_neighbors': [1, 3, 5, 7], 'algorithm': ['auto'], 'metric': ['manhattan', 'euclidean', 'minkowski']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [22]:
best_parameters = knn_gs.best_params_
print(best_parameters)

{'algorithm': 'auto', 'metric': 'manhattan', 'n_neighbors': 7}


In [27]:
resultKnn = knn(7, x_train0, x_test0, y_train0, y_test0)
resultKnn

42713.3133029687

In [240]:
random_forest_grid_param = {'n_estimators': [10, 20, 40, 50, 70, 80, 100, 120, 160, 200],
               'max_features': ['auto'],
               'max_depth': [3, 5, 7, 9, 15, 30, 40, 50],
               'min_samples_split': [2, 5, 8],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

In [ ]:
rf = RandomForestRegressor()
rf_gs = GridSearchCV(estimator=rf, param_grid=rf_grid_param, scoring="neg_mean_squared_error", n_jobs=-1)

In [ ]:
best_parameters = rf_gs.best_params_
print(best_parameters)

In [32]:
resultRF = randomForest(100, 5, x_train0, x_test0, y_train0, y_test0)
resultRF

40094.21483033612

In [33]:
resultRF2 = randomForest(100, 5, x_train4, x_test4, y_train4, y_test4)
resultRF2

78451.6331827559

In [ ]:
resultxgboost = xgboost()

In [241]:
xgb_grid_param = {
    'max_depth': [3, 5, 7], #[3,4,5,6,7,8,9], # 5 is good but takes too long in kaggle env
    'subsample': [0.6, 0.8], #[0.4,0.5,0.6,0.7,0.8,0.9,1.0],
    'colsample_bytree': [0.6, 0.8], #[0.5,0.6,0.7,0.8],
    'n_estimators': [100, 300], #[1000,2000,3000]
    'learning_rate': [0.1]}

In [242]:
XGB0 = XGBRegressor(learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.6, colsample_bytree=0.6)
XGB1 = XGBRegressor(learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.6, colsample_bytree=0.6)
XGB2 = XGBRegressor(learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.6, colsample_bytree=0.6)
XGB3 = XGBRegressor(learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.6, colsample_bytree=0.6)

XGB4 = XGBRegressor(learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.6, colsample_bytree=0.6)
XGB5 = XGBRegressor(learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.6, colsample_bytree=0.6)
XGB6 = XGBRegressor(learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.6, colsample_bytree=0.6)
XGB7 = XGBRegressor(learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.6, colsample_bytree=0.6)

In [34]:
XGB10 = XGBRegressor(learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.6, colsample_bytree=0.6)
XGB10.fit(x_train4, y_train4)
y_pred4 = XGB10.predict(x_test4)
np.sqrt(mean_squared_error(y_test4, y_pred4))

78000.7995068793

In [243]:
XGB0.fit(x_train0, y_train0)
y_pred0 = XGB0.predict(x_test0)
np.sqrt(mean_squared_error(y_test0, y_pred0))

40159.62519853466

In [244]:
XGB1.fit(x_train1, y_train1)
y_pred1 = XGB1.predict(x_test1)
np.sqrt(mean_squared_error(y_test1, y_pred1))

38472.98203745538

In [245]:
XGB2.fit(x_train2, y_train2)
y_pred2 = XGB2.predict(x_test2)
np.sqrt(mean_squared_error(y_test2, y_pred2))

37509.79562909647

In [246]:
XGB3.fit(x_train3, y_train3)
y_pred3 = XGB3.predict(x_test3)
np.sqrt(mean_squared_error(y_test3, y_pred3))

37848.29097872711

In [247]:
XGB4.fit(x_train4, y_train4)
y_pred4 = XGB4.predict(x_test4)
np.sqrt(mean_squared_error(y_test4, y_pred4))

78000.7995068793

In [248]:
XGB5.fit(x_train5, y_train5)
y_pred5 = XGB5.predict(x_test5)
np.sqrt(mean_squared_error(y_test5, y_pred5))

80684.44040608744

In [249]:
XGB6.fit(x_train6, y_train6)
y_pred6 = XGB6.predict(x_test6)
np.sqrt(mean_squared_error(y_test6, y_pred6))

76904.88962168366

In [250]:
XGB7.fit(x_train7, y_train7)
y_pred7 = XGB7.predict(x_test7)
np.sqrt(mean_squared_error(y_test7, y_pred7))

75984.16680540895

In [251]:
data_devices0_with_target_install_predict0 = XGB0.predict(data_devices0_with_target_install)
data_devices0_with_target_install_predict1 = XGB1.predict(data_devices0_with_target_install)
data_devices0_with_target_install_predict2 = XGB2.predict(data_devices0_with_target_install)
data_devices0_with_target_install_predict3 = XGB3.predict(data_devices0_with_target_install)

In [252]:
mean_data_devices0_with_target_install_predict = (data_devices0_with_target_install_predict0 + data_devices0_with_target_install_predict1 + data_devices0_with_target_install_predict2 + data_devices0_with_target_install_predict3) / 4

In [253]:
data_devices1_with_target_install_predict0 = XGB0.predict(data_devices1_with_target_install)
data_devices1_with_target_install_predict1 = XGB1.predict(data_devices1_with_target_install)
data_devices1_with_target_install_predict2 = XGB2.predict(data_devices1_with_target_install)
data_devices1_with_target_install_predict3 = XGB3.predict(data_devices1_with_target_install)

In [254]:
mean_data_devices1_with_target_install_predict = (data_devices1_with_target_install_predict0 + data_devices1_with_target_install_predict1 + data_devices1_with_target_install_predict2 + data_devices1_with_target_install_predict3) / 4

In [255]:
data_devices2_with_target_install_predict0 = XGB0.predict(data_devices2_with_target_install)
data_devices2_with_target_install_predict1 = XGB1.predict(data_devices2_with_target_install)
data_devices2_with_target_install_predict2 = XGB2.predict(data_devices2_with_target_install)
data_devices2_with_target_install_predict3 = XGB3.predict(data_devices2_with_target_install)

In [256]:
mean_data_devices2_with_target_install_predict = (data_devices2_with_target_install_predict0 + data_devices2_with_target_install_predict1 + data_devices2_with_target_install_predict2 + data_devices2_with_target_install_predict3) / 4

In [257]:
data_devices3_with_target_install_predict0 = XGB0.predict(data_devices3_with_target_install)
data_devices3_with_target_install_predict1 = XGB1.predict(data_devices3_with_target_install)
data_devices3_with_target_install_predict2 = XGB2.predict(data_devices3_with_target_install)
data_devices3_with_target_install_predict3 = XGB3.predict(data_devices3_with_target_install)

In [258]:
mean_data_devices3_with_target_install_predict = (data_devices3_with_target_install_predict0 + data_devices3_with_target_install_predict1 + data_devices3_with_target_install_predict2 + data_devices3_with_target_install_predict3) / 4

In [259]:
data_kaggle_with_target_install_predict0 = XGB0.predict(data_kaggle_with_target_install)
data_kaggle_with_target_install_predict1 = XGB1.predict(data_kaggle_with_target_install)
data_kaggle_with_target_install_predict2 = XGB2.predict(data_kaggle_with_target_install)
data_kaggle_with_target_install_predict3 = XGB3.predict(data_kaggle_with_target_install)

In [260]:
mean_data_kaggle_with_target_install_predict = (data_kaggle_with_target_install_predict0 + data_kaggle_with_target_install_predict1 + data_kaggle_with_target_install_predict2 + data_kaggle_with_target_install_predict3) / 4

In [261]:
data_devices0_with_target_auction_predict0 = XGB4.predict(data_devices0_with_target_auction)
data_devices0_with_target_auction_predict1 = XGB5.predict(data_devices0_with_target_auction)
data_devices0_with_target_auction_predict2 = XGB6.predict(data_devices0_with_target_auction)
data_devices0_with_target_auction_predict3 = XGB7.predict(data_devices0_with_target_auction)

In [262]:
mean_data_devices0_with_target_auction_predict = (data_devices0_with_target_auction_predict0 + data_devices0_with_target_auction_predict1 + data_devices0_with_target_auction_predict2 + data_devices0_with_target_auction_predict3) / 4

In [263]:
data_devices1_with_target_auction_predict0 = XGB4.predict(data_devices1_with_target_auction)
data_devices1_with_target_auction_predict1 = XGB5.predict(data_devices1_with_target_auction)
data_devices1_with_target_auction_predict2 = XGB6.predict(data_devices1_with_target_auction)
data_devices1_with_target_auction_predict3 = XGB7.predict(data_devices1_with_target_auction)

In [264]:
mean_data_devices1_with_target_auction_predict = (data_devices1_with_target_auction_predict0 + data_devices1_with_target_auction_predict1 + data_devices1_with_target_auction_predict2 + data_devices1_with_target_auction_predict3) / 4

In [265]:
data_devices2_with_target_auction_predict0 = XGB4.predict(data_devices2_with_target_auction)
data_devices2_with_target_auction_predict1 = XGB5.predict(data_devices2_with_target_auction)
data_devices2_with_target_auction_predict2 = XGB6.predict(data_devices2_with_target_auction)
data_devices2_with_target_auction_predict3 = XGB7.predict(data_devices2_with_target_auction)

In [266]:
mean_data_devices2_with_target_auction_predict = (data_devices2_with_target_auction_predict0 + data_devices2_with_target_auction_predict1 + data_devices2_with_target_auction_predict2 + data_devices2_with_target_auction_predict3) / 4

In [267]:
data_devices3_with_target_auction_predict0 = XGB4.predict(data_devices3_with_target_auction)
data_devices3_with_target_auction_predict1 = XGB5.predict(data_devices3_with_target_auction)
data_devices3_with_target_auction_predict2 = XGB6.predict(data_devices3_with_target_auction)
data_devices3_with_target_auction_predict3 = XGB7.predict(data_devices3_with_target_auction)

In [268]:
mean_data_devices3_with_target_auction_predict = (data_devices3_with_target_auction_predict0 + data_devices3_with_target_auction_predict1 + data_devices3_with_target_auction_predict2 + data_devices3_with_target_auction_predict3) / 4

In [269]:
data_kaggle_with_target_auction_predict0 = XGB4.predict(data_kaggle_with_target_auction)
data_kaggle_with_target_auction_predict1 = XGB5.predict(data_kaggle_with_target_auction)
data_kaggle_with_target_auction_predict2 = XGB6.predict(data_kaggle_with_target_auction)
data_kaggle_with_target_auction_predict3 = XGB7.predict(data_kaggle_with_target_auction)

In [270]:
mean_data_kaggle_with_target_auction_predict = (data_kaggle_with_target_auction_predict0 + data_kaggle_with_target_auction_predict1 + data_kaggle_with_target_auction_predict2 + data_kaggle_with_target_auction_predict3) / 4

In [271]:
devices0_target_install = {'ref_hash': devices0_with_target["ref_hash"], 'obj': mean_data_devices0_with_target_install_predict}
df_devices0_target_install = pd.DataFrame(data=devices0_target_install)
devices1_target_install = {'ref_hash': devices1_with_target["ref_hash"], 'obj': mean_data_devices1_with_target_install_predict}
df_devices1_target_install = pd.DataFrame(data=devices1_target_install)
devices2_target_install = {'ref_hash': devices2_with_target["ref_hash"], 'obj': mean_data_devices2_with_target_install_predict}
df_devices2_target_install = pd.DataFrame(data=devices2_target_install)
devices3_target_install = {'ref_hash': devices3_with_target["ref_hash"], 'obj': mean_data_devices3_with_target_install_predict}
df_devices3_target_install = pd.DataFrame(data=devices3_target_install)
kaggle_target_install = {'ref_hash': kaggle_with_target["ref_hash"], 'obj': mean_data_kaggle_with_target_install_predict}
df_kaggle_target_install = pd.DataFrame(data=kaggle_target_install)

devices0_target_auction = {'ref_hash': devices0_with_target["ref_hash"], 'obj': mean_data_devices0_with_target_auction_predict}
df_devices0_target_auction = pd.DataFrame(data=devices0_target_auction)
devices1_target_auction = {'ref_hash': devices1_with_target["ref_hash"], 'obj': mean_data_devices1_with_target_auction_predict}
df_devices1_target_auction = pd.DataFrame(data=devices1_target_auction)
devices2_target_auction = {'ref_hash': devices2_with_target["ref_hash"], 'obj': mean_data_devices2_with_target_auction_predict}
df_devices2_target_auction = pd.DataFrame(data=devices2_target_auction)
devices3_target_auction = {'ref_hash': devices3_with_target["ref_hash"], 'obj': mean_data_devices3_with_target_auction_predict}
df_devices3_target_auction = pd.DataFrame(data=devices3_target_auction)
kaggle_target_auction = {'ref_hash': kaggle_with_target["ref_hash"], 'obj': mean_data_kaggle_with_target_auction_predict}
df_kaggle_target_auction = pd.DataFrame(data=kaggle_target_auction)

In [272]:
df_devices0_target_install['ref_hash'] = df_devices0_target_install['ref_hash'].astype(str) + '_sc'
df_devices1_target_install['ref_hash'] = df_devices1_target_install['ref_hash'].astype(str) + '_sc'
df_devices2_target_install['ref_hash'] = df_devices2_target_install['ref_hash'].astype(str) + '_sc'
df_devices3_target_install['ref_hash'] = df_devices3_target_install['ref_hash'].astype(str) + '_sc'
df_kaggle_target_install['ref_hash'] = df_kaggle_target_install['ref_hash'].astype(str) + '_sc'

df_devices0_target_auction['ref_hash'] = df_devices0_target_auction['ref_hash'].astype(str) + '_st'
df_devices1_target_auction['ref_hash'] = df_devices1_target_auction['ref_hash'].astype(str) + '_st'
df_devices2_target_auction['ref_hash'] = df_devices2_target_auction['ref_hash'].astype(str) + '_st'
df_devices3_target_auction['ref_hash'] = df_devices3_target_auction['ref_hash'].astype(str) + '_st'
df_kaggle_target_auction['ref_hash'] = df_kaggle_target_auction['ref_hash'].astype(str) + '_st'

In [273]:
target = target.drop('obj', 1)
target_kaggle = target.merge(df_kaggle_target_install, on='ref_hash', how='left')
target_kaggle = target_kaggle.merge(df_devices0_target_install, on='ref_hash', how='left')
target_kaggle = target_kaggle.merge(df_devices1_target_install, on='ref_hash', how='left')
target_kaggle = target_kaggle.merge(df_devices2_target_install, on='ref_hash', how='left')
target_kaggle = target_kaggle.merge(df_devices3_target_install, on='ref_hash', how='left')
target_kaggle = target_kaggle.merge(df_kaggle_target_auction, on='ref_hash', how='left')
target_kaggle = target_kaggle.merge(df_devices0_target_auction, on='ref_hash', how='left')
target_kaggle = target_kaggle.merge(df_devices1_target_auction, on='ref_hash', how='left')
target_kaggle = target_kaggle.merge(df_devices2_target_auction, on='ref_hash', how='left')
target_kaggle = target_kaggle.merge(df_devices3_target_auction, on='ref_hash', how='left')
target_kaggle

,ref_hash,obj_x,obj_y,obj_x,obj_y,obj_x,obj_y,obj_x,obj_y,obj_x,obj_y
0,1000169251625791246_sc,245600.375000,240209.718750,253298.468750,254873.187500,251040.906250,NaN,NaN,NaN,NaN,NaN
1,1000169251625791246_st,NaN,NaN,NaN,NaN,NaN,37900.398438,98445.578125,85310.968750,93957.343750,88797.093750
2,1000395625957344683_sc,NaN,253023.296875,255583.437500,255185.500000,NaN,NaN,NaN,NaN,NaN,NaN
3,1000395625957344683_st,NaN,NaN,NaN,NaN,NaN,NaN,134321.609375,141146.062500,112196.031250,NaN
4,1003027494996471685_sc,242324.593750,247304.546875,241281.625000,242498.265625,250798.953125,NaN,NaN,NaN,NaN,NaN
5,1003027494996471685_st,NaN,NaN,NaN,NaN,NaN,120856.859375,168486.687500,131174.281250,143974.718750,175554.218750
6,1006670001679961544_sc,NaN,NaN,255450.343750,254073.156250,252633.781250,NaN,NaN,NaN,NaN,NaN
7,1006670001679961544_st,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230166.625000,226862.062500,225310.781250
8,1007573308966476713_sc,NaN,253717.031250,246952.250000,244499.468750,244496.093750,NaN,NaN,NaN,NaN,NaN
9,1007573308966476713_st,NaN,NaN,NaN,NaN,NaN,NaN,132981.687500,153998.843750,109883.976562,129795.437500


In [283]:
mean_prediction = target_kaggle.mean(axis = 1, skipna = True)

1528

In [285]:
final_target_kaggle = {'ref_hash': target_kaggle["ref_hash"], 'obj': mean_prediction}
df_target_kaggle = pd.DataFrame(data=final_target_kaggle)
df_target_kaggle = df_target_kaggle.fillna(129600)
df_target_kaggle

,ref_hash,obj
0,1000169251625791246_sc,249004.531250
1,1000169251625791246_st,80882.273438
2,1000395625957344683_sc,254597.421875
3,1000395625957344683_st,129221.242188
4,1003027494996471685_sc,244841.593750
5,1003027494996471685_st,148009.343750
6,1006670001679961544_sc,254052.421875
7,1006670001679961544_st,227446.500000
8,1007573308966476713_sc,247416.218750
9,1007573308966476713_st,131664.984375


In [286]:
df_target_kaggle.to_csv('setSubmit.csv', index=False)

In [99]:
resultAda = adaboost(100, 0.5, x_train0, x_test0, y_train0, y_test0)

In [100]:
resultAda

47433.49071917786

In [103]:
xgboost = XGBRegressor(learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.6, colsample_bytree=0.6)

In [108]:
resultbag = bagging(xgboost, 100, 0.1, 5, True, x_train0, x_test0, y_train0, y_test0)

In [109]:
resultbag

40202.578557589455

In [65]:
xgb1 = XGBRegressor()

In [66]:
xgb_gs = GridSearchCV(estimator=xgb1, param_grid=xgb_grid_param, scoring="neg_mean_squared_error", cv=3, n_jobs=1)

In [67]:
xgb_gs.fit(x_train0, y_train0)

KeyboardInterrupt: 